In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
dev= pd.read_csv('devices.csv')
tr1= pd.read_csv('transactions_1.csv')
tr2= pd.read_csv('transactions_2.csv')
tr3= pd.read_csv('transactions_3.csv')
users = pd.read_csv('users.csv')
notif = pd.read_csv('notifications.csv')
trans= pd.concat([tr1, tr2, tr3])#all transactions in one dataframe

#users table

In [0]:
users['brands']=dev['brand']#h sthlh brands apo ta devices is added to users table
#ginontai ta unknown stoixeia tyxaia Android, Apple
import random
def change_unknown_device(brand):
  if  brand== 'Unknown':
    return random.sample(['Android', 'Apple'], 1)[0]
  else:
    return brand
users['brands']=users['brands'].apply(change_unknown_device)


In [0]:
users['age']=2019 - users['birth_year']#sthlh age anti gia birth_year

In [0]:
users['plan']=users['plan'].apply(lambda x: 'PAID' if x!="STANDARD" else x)#h sthlh pairnei pleon standard h paid stoixeia

In [0]:
#handling dates in days
users.created_date=pd.to_datetime(users.created_date)
#users['today']=np.datetime64('2019-06-01')
users['account_duration'] = (np.datetime64('2019-06-01') - users['created_date']).dt.days
temp_users = users

In [0]:
#users['month']=pd.DatetimeIndex(users['created_date']).month
#users['year']=pd.DatetimeIndex(users['created_date']).year
#users['duration']=users.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

#transactions table
-------------------

In [0]:
trans=trans[trans['transactions_state']=='COMPLETED']#kratame mono ta completed transactions

In [698]:
#tempo table me hmeromhnies twn transaction 
#Oi hmeromhnies ypologizontai se meres apo thn 1/6
tempo=trans[['user_id','created_date']]
tempo.sort_values(['user_id','created_date'],inplace=True)
tempo['created_date']=pd.to_datetime(tempo['created_date'])
tempo['days_from_transaction']=(np.datetime64('2019-06-01') - tempo['created_date']).dt.days
tempo.drop(columns=['created_date'], inplace=True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [0]:
#uni table exei tis unique dates poy exei kanei ka8e user transaction
uni = tempo.drop_duplicates(["user_id","days_from_transaction"])


In [0]:
#ypologizontai o mean tvn diaforwn kathe transaction, kai h max diafora transaction gia kathe user
mean_period_within_trans=uni.groupby(['user_id']).agg(mean_period_within_transactions=('days_from_transaction', lambda x: x.diff(periods=-1).mean()))
max_period_within_trans=uni.groupby(['user_id']).agg(max_period_within_transactions=('days_from_transaction', lambda x: x.diff(periods=-1).max()))
       

In [0]:
users=pd.merge(users , mean_period_within_trans, how='left', left_on='user_id', right_index=True)

In [0]:
users=pd.merge(users , max_period_within_trans, how='left', left_on='user_id', right_index=True)

In [0]:
#ypologizetai poses unique dates exei kanei transactions kathe user
unique_days_of_trans=tempo.groupby(['user_id']).agg(unique_days_of_transactions=('days_from_transaction', 'nunique'))

In [0]:
users=pd.merge(users , unique_days_of_trans, how='left', left_on='user_id', right_index=True)

In [0]:
#ypolofizetai to std, var twn dates of transactions gia ka8e user
std_var_trans=tempo.groupby('user_id').agg(std_of_transactions=('days_from_transaction','std'),var_of_transactions=('days_from_transaction','var'))


In [0]:
users=pd.merge(users , std_var_trans, how='left', left_on='user_id', right_index=True)

In [0]:
number_of_trans=pd.DataFrame(trans.user_id.value_counts())#amount of transactions per user
number_of_trans.rename(columns={'user_id':'total_amount_of_transactions'},inplace=True)
users=pd.merge(users , number_of_trans, how='left', left_on='user_id', right_index=True)#h sthlg amount_of_transactions is joined sto users

In [0]:
trans_out=trans[trans['direction']=='OUTBOUND']
trans_in=trans[trans['direction']=='INBOUND']

In [0]:
usd_per_user_out = pd.pivot_table(data=trans_out,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
usd_per_user_out.columns=['amount_of_transactions_out', 'min_usd_out', 'max_usd_out', 'mean_usd_out', 'sum_usd_out']
users=pd.merge(users , usd_per_user_out, how='left', left_on='user_id', right_index=True)

In [0]:
usd_per_user_in = pd.pivot_table(data=trans_in,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
usd_per_user_in.columns=['amount_of_transactions_in', 'min_usd_in', 'max_usd_in', 'mean_usd_in', 'sum_usd_in']
users=pd.merge(users , usd_per_user_in, how='left', left_on='user_id', right_index=True)

In [0]:
#pd.crosstab(trans.transactions_type, trans.ea_merchant_country)

In [0]:
#pd.crosstab(trans.transactions_type, trans.ea_cardholderpresence)

In [0]:
trans_transactions_type_out = pd.crosstab(trans_out['user_id'], columns=trans_out['transactions_type'])
trans_transactions_type_in = pd.crosstab(trans_in['user_id'], columns=trans_in['transactions_type'])
trans_transactions_type_out.columns=['count_type_atm', 'count_type_card_payment','count_type_exchange','count_type_fee', 'count_type_tax', 'count_type_transfer_out']
trans_transactions_type_in.columns=['count_type_card_refund', 'count_type_cashback','count_type_refund','count_type_topup','count_type_transfer_in']

In [0]:
users=pd.merge(users , trans_transactions_type_out, how='left', left_on='user_id', right_index=True)

In [0]:
users=pd.merge(users , trans_transactions_type_in, how='left', left_on='user_id', right_index=True)

In [0]:
#trans['month']=pd.DatetimeIndex(trans['created_date']).month#created month and year as columns
#trans['year']=pd.DatetimeIndex(trans['created_date']).year


In [0]:
trans_sorted=trans.sort_values(['created_date'],ascending=False)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
lasttrans = trans_sorted.drop_duplicates('user_id').sort_index()
lasttrans=lasttrans[['user_id','created_date']]

In [0]:
#handling dates in days
lasttrans['created_date']=pd.to_datetime(lasttrans['created_date'])
#lasttrans['today']=np.datetime64('2019-06-01')
lasttrans['last_transaction'] = (np.datetime64('2019-06-01') - lasttrans['created_date']).dt.days

In [0]:
#lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
#lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
#lasttrans['months_from_last_transaction']=lasttrans.apply(lambda x: 17-x['month'] if x['year']==2018 else 5-x['month'], axis=1)

In [0]:
users=pd.merge(users , lasttrans, how='left', left_on='user_id', right_on='user_id')

In [0]:
trans_sorted_asc=trans.sort_values(['created_date'],ascending=True)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
firsttrans = trans_sorted_asc.drop_duplicates('user_id').sort_index()
firsttrans=firsttrans[['user_id','created_date']]

In [0]:
#handling dates in days
firsttrans['created_date']=pd.to_datetime(firsttrans['created_date'])
#firsttrans['today']=np.datetime64('2019-06-01')
firsttrans['first_transaction'] = (np.datetime64('2019-06-01')-firsttrans['created_date']).dt.days

In [0]:
#lasttrans['month']=pd.DatetimeIndex(lasttrans['created_date']).month
#lasttrans['year']=pd.DatetimeIndex(lasttrans['created_date']).year
#lasttrans['months_from_first_transaction']=lasttrans.apply(lambda x: 18-x['month'] if x['year']==2018 else 6-x['month'], axis=1)

In [0]:
users=pd.merge(users , firsttrans, how='left', left_on='user_id', right_on='user_id')

In [725]:
users.sample(4)

,user_id,birth_year,country,city,created_date_x,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brands,age,account_duration,mean_period_within_transactions,max_period_within_transactions,unique_days_of_transactions,std_of_transactions,var_of_transactions,total_amount_of_transactions,amount_of_transactions_out,min_usd_out,max_usd_out,mean_usd_out,sum_usd_out,amount_of_transactions_in,min_usd_in,max_usd_in,mean_usd_in,sum_usd_in,count_type_atm,count_type_card_payment,count_type_exchange,count_type_fee,count_type_tax,count_type_transfer_out,count_type_card_refund,count_type_cashback,count_type_refund,count_type_topup,count_type_transfer_in,created_date_y,last_transaction,created_date,first_transaction
4495,user_4495,1982,RO,Chiajna,2018-01-09 23:39:27.320943,0,STANDARD,1.0,1.0,66,0,0,Apple,37,507,6.500000,12.0,3.0,6.765107,45.766667,6.0,3.0,0.18,6.62,3.520000,10.56,3.0,0.18,8.95,4.713333,14.14,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,2019-04-20 15:52:21.393943,41.0,2019-04-07 10:15:49.918943,54.0
3317,user_3317,1991,CZ,Uhersky Brod,2018-05-23 21:33:28.183284,0,STANDARD,1.0,1.0,11,0,0,Android,28,373,NaN,NaN,1.0,0.000000,0.000000,2.0,1.0,3.31,3.31,3.310000,3.31,1.0,9.92,9.92,9.920000,9.92,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2018-05-23 21:47:52.829284,373.0,2018-05-23 21:36:10.293284,373.0
1101,user_1101,1973,PT,Lisboa,2018-04-16 04:47:46.432240,0,PAID,1.0,1.0,110,0,0,Apple,46,410,1.634454,36.0,239.0,81.181647,6590.459864,1131.0,835.0,0.01,4000.00,93.893234,78400.85,296.0,0.01,4255.32,168.137095,49768.58,4.0,480.0,148.0,2.0,0.0,201.0,0.0,281.0,1.0,14.0,0.0,2019-05-10 19:26:21.349240,21.0,2018-04-16 06:46:58.519240,410.0
624,user_624,1957,NL,Abb,2018-06-09 20:57:34.008448,0,STANDARD,1.0,1.0,0,0,0,Apple,62,356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN


In [0]:
users['engagement_duration']=users['first_transaction'] - users['last_transaction'] + 1

In [727]:
users.columns

Index(['user_id', 'birth_year', 'country', 'city', 'created_date_x',
       'user_settings_crypto_unlocked', 'plan',
       'attributes_notifications_marketing_push',
       'attributes_notifications_marketing_email', 'num_contacts',
       'num_referrals', 'num_successful_referrals', 'brands', 'age',
       'account_duration', 'mean_period_within_transactions',
       'max_period_within_transactions', 'unique_days_of_transactions',
       'std_of_transactions', 'var_of_transactions',
       'total_amount_of_transactions', 'amount_of_transactions_out',
       'min_usd_out', 'max_usd_out', 'mean_usd_out', 'sum_usd_out',
       'amount_of_transactions_in', 'min_usd_in', 'max_usd_in', 'mean_usd_in',
       'sum_usd_in', 'count_type_atm', 'count_type_card_payment',
       'count_type_exchange', 'count_type_fee', 'count_type_tax',
       'count_type_transfer_out', 'count_type_card_refund',
       'count_type_cashback', 'count_type_refund', 'count_type_topup',
       'count_type_transfer_in'

In [0]:
users['days_passed_for_first_transaction']= users['account_duration']- users['first_transaction'] 

In [729]:
users.sample()

,user_id,birth_year,country,city,created_date_x,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brands,age,account_duration,mean_period_within_transactions,max_period_within_transactions,unique_days_of_transactions,std_of_transactions,var_of_transactions,total_amount_of_transactions,amount_of_transactions_out,min_usd_out,max_usd_out,mean_usd_out,sum_usd_out,amount_of_transactions_in,min_usd_in,max_usd_in,mean_usd_in,sum_usd_in,count_type_atm,count_type_card_payment,count_type_exchange,count_type_fee,count_type_tax,count_type_transfer_out,count_type_card_refund,count_type_cashback,count_type_refund,count_type_topup,count_type_transfer_in,created_date_y,last_transaction,created_date,first_transaction,engagement_duration,days_passed_for_first_transaction
6838,user_6838,1997,FR,strasbourg,2018-07-21 23:48:29.872378,0,STANDARD,1.0,1.0,0,0,0,Android,22,314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaT,NaN,NaN,NaN


In [0]:
users['transactions_frequency']=users['total_amount_of_transactions'] / users['engagement_duration']

#Notifications
________________________________________________________


In [0]:
notif_sent=notif[notif['status']=='SENT']


In [0]:
amount_notif_sent=pd.crosstab(notif_sent['user_id'], columns=[notif_sent['channel'],notif_sent['reason']])
amount_notif_sent.columns=['_'.join(col).lower() for col in amount_notif_sent.columns.values]

In [0]:
users=pd.merge(users, amount_notif_sent, how='left', left_on='user_id', right_index=True)


###notifications table analysis under process

In [0]:
#pd.crosstab(notif['reason'], columns=notif['status'])

In [0]:
#pd.crosstab(notif['user_id'], columns=[notif['channel'], notif['status'], notif['reason']]).head(5)

In [0]:
#notif[notif['user_id']=='user_5734'].sort_values('created_date')#Z

In [0]:
#users[users['user_id']=='user_12544']

In [0]:
#=pd.crosstab(notif['user_id'], columns= notif['reason'])
#tepm2.sample(5)

In [0]:
#=pd.merge(users, tepm2, left_on='user_id', right_index=True )

In [0]:
#=temp[['plan','BLACK_FRIDAY','BLUE_TUESDAY','ENGAGEMENT_SPLIT_BILL_RESTAURANT','ENGAGEMENT_SPLIT_BILL_RESTAURANT','INVEST_IN_GOLD','JOINING_ANNIVERSARY','LOST_CARD_ORDER','MADE_MONEY_REQUEST_NOT_SPLIT_BILL','METAL_RESERVE_PLAN','NO_INITIAL_CARD_ORDER','NO_INITIAL_CARD_USE','ONBOARDING_TIPS_ACTIVATED_USERS','PROMO','PROMO_CARD_ORDER','REENGAGEMENT_ACTIVE_FUNDS','SILVER_ENGAGEMENT_FEES_SAVED','SILVER_ENGAGEMENT_INACTIVE_CARD','WELCOME_BACK']]

In [0]:
#temp[temp['plan']=='PAID'].sample(50)

#Dropping and encoding users table

In [0]:
#dropping columns we don't need
data=users.drop(columns=['birth_year','city','created_date_x','attributes_notifications_marketing_push','attributes_notifications_marketing_email','num_referrals','num_successful_referrals','created_date_y','created_date'])


#'year_x','month_x','month_y','year_y','month','year'

In [743]:
data.columns

Index(['user_id', 'country', 'user_settings_crypto_unlocked', 'plan',
       'num_contacts', 'brands', 'age', 'account_duration',
       'mean_period_within_transactions', 'max_period_within_transactions',
       'unique_days_of_transactions', 'std_of_transactions',
       'var_of_transactions', 'total_amount_of_transactions',
       'amount_of_transactions_out', 'min_usd_out', 'max_usd_out',
       'mean_usd_out', 'sum_usd_out', 'amount_of_transactions_in',
       'min_usd_in', 'max_usd_in', 'mean_usd_in', 'sum_usd_in',
       'count_type_atm', 'count_type_card_payment', 'count_type_exchange',
       'count_type_fee', 'count_type_tax', 'count_type_transfer_out',
       'count_type_card_refund', 'count_type_cashback', 'count_type_refund',
       'count_type_topup', 'count_type_transfer_in', 'last_transaction',
       'first_transaction', 'engagement_duration',
       'days_passed_for_first_transaction', 'transactions_frequency',
       'email_engagement_split_bill_restaurant', 'email_i

In [0]:
data.isnull().sum()# ta nan tha ginoyn 0, afoy einai eite counts eite sum klp,kai 
datatest=data.fillna(0)#dhmioyrgeitai o datatest gia xrhsh se modela

In [0]:
#column plan  encoding sto dataset
plan_dict = {'STANDARD': 0, 'PAID': 1}
datatest['plan'] = datatest.plan.map(plan_dict)



In [0]:
datatest.drop('country', axis=1, inplace=True)
#gia thn wra petaw ta countries mexri na doyme pws ua ta xeiristoume

In [0]:
#column brands encoding sto dataset
datatest=pd.get_dummies(datatest)

In [0]:
#dhmioyrgoyntai oi pinakes pou exw anevasei
datatest.to_csv('latest_for_testing.csv',index=False, sep=';')
data.to_csv('latest_from_users.csv',index=False, sep=';')
users.to_csv('latest_users_feat.csv',index=False, sep=';')

In [749]:
datatest[datatest['transactions_frequency']==np.inf]

,user_settings_crypto_unlocked,plan,num_contacts,age,account_duration,mean_period_within_transactions,max_period_within_transactions,unique_days_of_transactions,std_of_transactions,var_of_transactions,total_amount_of_transactions,amount_of_transactions_out,min_usd_out,max_usd_out,mean_usd_out,sum_usd_out,amount_of_transactions_in,min_usd_in,max_usd_in,mean_usd_in,sum_usd_in,count_type_atm,count_type_card_payment,count_type_exchange,count_type_fee,count_type_tax,count_type_transfer_out,count_type_card_refund,count_type_cashback,count_type_refund,count_type_topup,count_type_transfer_in,last_transaction,first_transaction,engagement_duration,days_passed_for_first_transaction,transactions_frequency,email_engagement_split_bill_restaurant,email_invest_in_gold,email_lost_card_order,...,user_id_user_9965,user_id_user_9966,user_id_user_9967,user_id_user_9968,user_id_user_9969,user_id_user_997,user_id_user_9970,user_id_user_9971,user_id_user_9972,user_id_user_9973,user_id_user_9974,user_id_user_9975,user_id_user_9976,user_id_user_9977,user_id_user_9978,user_id_user_9979,user_id_user_998,user_id_user_9980,user_id_user_9981,user_id_user_9982,user_id_user_9983,user_id_user_9984,user_id_user_9985,user_id_user_9986,user_id_user_9987,user_id_user_9988,user_id_user_9989,user_id_user_999,user_id_user_9990,user_id_user_9991,user_id_user_9992,user_id_user_9993,user_id_user_9994,user_id_user_9995,user_id_user_9996,user_id_user_9997,user_id_user_9998,user_id_user_9999,brands_Android,brands_Apple


In [750]:
datatest.head()

,user_settings_crypto_unlocked,plan,num_contacts,age,account_duration,mean_period_within_transactions,max_period_within_transactions,unique_days_of_transactions,std_of_transactions,var_of_transactions,total_amount_of_transactions,amount_of_transactions_out,min_usd_out,max_usd_out,mean_usd_out,sum_usd_out,amount_of_transactions_in,min_usd_in,max_usd_in,mean_usd_in,sum_usd_in,count_type_atm,count_type_card_payment,count_type_exchange,count_type_fee,count_type_tax,count_type_transfer_out,count_type_card_refund,count_type_cashback,count_type_refund,count_type_topup,count_type_transfer_in,last_transaction,first_transaction,engagement_duration,days_passed_for_first_transaction,transactions_frequency,email_engagement_split_bill_restaurant,email_invest_in_gold,email_lost_card_order,...,user_id_user_9965,user_id_user_9966,user_id_user_9967,user_id_user_9968,user_id_user_9969,user_id_user_997,user_id_user_9970,user_id_user_9971,user_id_user_9972,user_id_user_9973,user_id_user_9974,user_id_user_9975,user_id_user_9976,user_id_user_9977,user_id_user_9978,user_id_user_9979,user_id_user_998,user_id_user_9980,user_id_user_9981,user_id_user_9982,user_id_user_9983,user_id_user_9984,user_id_user_9985,user_id_user_9986,user_id_user_9987,user_id_user_9988,user_id_user_9989,user_id_user_999,user_id_user_9990,user_id_user_9991,user_id_user_9992,user_id_user_9993,user_id_user_9994,user_id_user_9995,user_id_user_9996,user_id_user_9997,user_id_user_9998,user_id_user_9999,brands_Android,brands_Apple
0,0,0,3,27,263,1.522013,18.0,160.0,64.059353,4103.600767,520.0,357.0,0.58,510.64,10.167367,3629.75,163.0,8.51,510.64,20.216748,3295.33,0.0,333.0,5.0,0.0,0.0,19.0,1.0,0.0,0.0,158.0,4.0,21.0,263.0,243.0,0.0,2.139918,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,75,34,505,6.676471,79.0,69.0,104.200464,10857.736728,133.0,106.0,0.94,851.06,61.226698,6490.03,27.0,1.70,851.06,239.463333,6465.51,4.0,78.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,24.0,3.0,27.0,481.0,455.0,24.0,0.292308,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,71,23,183,2.238095,24.0,64.0,25.899183,670.767664,268.0,225.0,0.04,194.38,6.901511,1552.84,43.0,4.26,97.87,27.233953,1171.06,0.0,97.0,0.0,0.0,0.0,128.0,0.0,0.0,0.0,37.0,6.0,42.0,183.0,142.0,0.0,1.887324,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,219,38,468,5.544304,104.0,80.0,104.476260,10915.289003,140.0,126.0,1.87,263.83,27.991667,3526.95,14.0,42.55,510.64,248.024286,3472.34,0.0,120.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,13.0,1.0,22.0,460.0,439.0,8.0,0.318907,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,27,508,9.484848,71.0,34.0,66.093345,4368.330204,50.0,41.0,1.50,92.00,13.691951,561.37,9.0,10.00,200.00,46.673333,420.06,0.0,37.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,0.0,136.0,449.0,314.0,59.0,0.159236,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [0]:
#dhmioyrgia X, y pinakwn gia na etoimasia toy train_test_split
#X = us_1_en.drop(columns=['plan']).values
#X_columns = us_1_en.drop(columns=['plan']).columns
#y = us_1_en['plan'].values

#NOTIF
---------------------------------

In [0]:
notif.created_date=pd.to_datetime(notif.created_date)
notif['days'] = (np.datetime64('2019-06-01') - notif['created_date']).dt.days

In [773]:
#a[0] = FAILED notifications per quarter year
#a[1] = SENT notifications per quarter year
a = [[0,0,0,0,0,0],[0,0,0,0,0,0]]
for x in range(0,len(notif)):
  if(notif['status'][x] != 'SENT'):
    a[0][notif['days'][x]//90] += 1
  else:
    a[1][notif['days'][x]//90] += 1
print(a) #0 -> nearest, 5-> oldest

[[2061, 12865, 3658, 3509, 3829, 89], [6747, 36130, 18498, 5937, 2868, 1513]]


In [774]:
a[0][2]

3658

In [755]:
count = 0
for x in range (0,6):
  for y in range (0,2):
    count += a[y][x]
print(count)

97704


In [756]:
len(notif)

97704

In [757]:
len(notif.query('status == "SENT" '))

71693

#Churn
----------------------------------

In [0]:
def func(churn):
    result = 0
    if churn['total_amount_of_transactions'] == 0:
        result= 1
    if (churn['std_of_transactions']==0):
        result= 1
    if (churn['last_transaction'] - 15) > (2 * churn['max_period_within_transactions']):
        result=1
    return result

In [0]:
users['churned_users']=users.apply(func, axis=1)

In [760]:
users.shape

(15544, 70)

#Trans
------------------------------------

In [0]:
trans.created_date=pd.to_datetime(trans.created_date)
trans['days'] = (np.datetime64('2019-06-01') - trans['created_date']).dt.days

In [762]:
trans.sample(1)

,transaction_id,transactions_type,transactions_currency,amount_usd,transactions_state,ea_cardholderpresence,ea_merchant_mcc,ea_merchant_city,ea_merchant_country,direction,user_id,created_date,days
46403,transaction_1546403,TRANSFER,RON,0.02,COMPLETED,NaN,NaN,NaN,NaN,OUTBOUND,user_6264,2019-04-15 06:00:48.872931,46


#crosstabs(CHURN, NOTIFICATIONS)
--------------------------------------

In [0]:
def my_func(trans, user, quarters):  
  trans=trans[trans['transactions_state']=='COMPLETED']#kratame mono ta completed transactions
  #tempo table me hmeromhnies twn transaction 
  #Oi hmeromhnies ypologizontai se meres apo thn 1/6
  tempo=trans[['user_id','created_date']]
  tempo.sort_values(['user_id','created_date'],inplace=True)
  tempo['created_date']=pd.to_datetime(tempo['created_date'])
  tempo['days_from_transaction']=(np.datetime64('2019-06-01') - tempo['created_date']).dt.days
  tempo.drop(columns=['created_date'], inplace=True)
  #uni table exei tis unique dates poy exei kanei ka8e user transaction
  uni = tempo.drop_duplicates(["user_id","days_from_transaction"])
  #ypologizontai o mean tvn diaforwn kathe transaction, kai h max diafora transaction gia kathe user
  mean_period_within_trans=uni.groupby(['user_id']).agg(mean_period_within_transactions=('days_from_transaction', lambda x: x.diff(periods=-1).mean()))
  max_period_within_trans=uni.groupby(['user_id']).agg(max_period_within_transactions=('days_from_transaction', lambda x: x.diff(periods=-1).max()))
  user=pd.merge(user , mean_period_within_trans, how='left', left_on='user_id', right_index=True)
  user=pd.merge(user , max_period_within_trans, how='left', left_on='user_id', right_index=True)
  #ypologizetai poses unique dates exei kanei transactions kathe user
  unique_days_of_trans=tempo.groupby(['user_id']).agg(unique_days_of_transactions=('days_from_transaction', 'nunique'))
  user=pd.merge(user , unique_days_of_trans, how='left', left_on='user_id', right_index=True)
  #ypolofizetai to std, var twn dates of transactions gia ka8e user
  std_var_trans=tempo.groupby('user_id').agg(std_of_transactions=('days_from_transaction','std'),var_of_transactions=('days_from_transaction','var'))
  user=pd.merge(user , std_var_trans, how='left', left_on='user_id', right_index=True)
  number_of_trans=pd.DataFrame(trans.user_id.value_counts())#amount of transactions per user
  number_of_trans.rename(columns={'user_id':'total_amount_of_transactions'},inplace=True)
  user=pd.merge(user , number_of_trans, how='left', left_on='user_id', right_index=True)#h sthlg amount_of_transactions is joined sto users
  trans_out=trans[trans['direction']=='OUTBOUND']
  trans_in=trans[trans['direction']=='INBOUND']
  usd_per_user_out = pd.pivot_table(data=trans_out,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
  usd_per_user_out.columns=['amount_of_transactions_out', 'min_usd_out', 'max_usd_out', 'mean_usd_out', 'sum_usd_out']
  user=pd.merge(user , usd_per_user_out, how='left', left_on='user_id', right_index=True)
  usd_per_user_in = pd.pivot_table(data=trans_in,aggfunc=['count','min','max','mean','sum'], index='user_id',  values='amount_usd')
  usd_per_user_in.columns=['amount_of_transactions_in', 'min_usd_in', 'max_usd_in', 'mean_usd_in', 'sum_usd_in']
  user=pd.merge(user , usd_per_user_in, how='left', left_on='user_id', right_index=True)
  trans_transactions_type_out = pd.crosstab(trans_out['user_id'], columns=trans_out['transactions_type'])
  trans_transactions_type_in = pd.crosstab(trans_in['user_id'], columns=trans_in['transactions_type'])
  trans_transactions_type_out.columns=['count_type_atm', 'count_type_card_payment','count_type_exchange','count_type_fee', 'count_type_tax', 'count_type_transfer_out']
  trans_transactions_type_in.columns=['count_type_card_refund', 'count_type_cashback','count_type_refund','count_type_topup','count_type_transfer_in']
  user=pd.merge(user , trans_transactions_type_out, how='left', left_on='user_id', right_index=True)
  user=pd.merge(user , trans_transactions_type_in, how='left', left_on='user_id', right_index=True)
  trans_sorted=trans.sort_values(['created_date'],ascending=False)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
  lasttrans = trans_sorted.drop_duplicates('user_id').sort_index()
  lasttrans=lasttrans[['user_id','created_date']]
  #handling dates in days
  lasttrans['created_date']=pd.to_datetime(lasttrans['created_date'])
  #lasttrans['today']=np.datetime64('2019-06-01')
  lasttrans['last_transaction'] = (np.datetime64(quarters) - lasttrans['created_date']).dt.days
  user=pd.merge(user , lasttrans, how='left', left_on='user_id', right_on='user_id')
  trans_sorted_asc=trans.sort_values(['created_date'],ascending=True)#trans is sorted, duplicates user_id entries are removed and remain only latest transactions 
  firsttran = trans_sorted_asc.drop_duplicates('user_id').sort_index()
  firsttran=firsttran[['user_id','created_date']]
  
 
  #handling dates in days
  firsttran['created_date']=pd.to_datetime(firsttran['created_date'])
  #firsttran['today']=np.datetime64('2019-06-01')
  firsttran['first_transaction'] = (np.datetime64(quarters)-firsttran['created_date']).dt.days
  user=pd.merge(user , firsttran, how='left', left_on='user_id', right_on='user_id')
  user['engagement_duration']=user['first_transaction'] - user['last_transaction'] + 1
  user['days_passed_for_first_transaction']= user['account_duration']- user['first_transaction'] 
  user['transactions_frequency']=user['total_amount_of_transactions'] / user['engagement_duration']
  return user

In [764]:
#churn users per quarter year
days = np.max(trans['days']) - 90
quarters = ['2018-03-01','2018-06-01','2018-09-01','2018-12-01','2019-03-01','2019-06-01']
c = [0,0,0,0,0,0]
print (days)
raw_users = temp_users
temp_trans = trans
print( temp_trans.shape)
for q in range (0,6):
  raw_users = temp_users
  temp = temp_trans[temp_trans['days'] >= days]
  
  g = my_func(temp_trans, raw_users, quarters[q])

  #temp_trans = temp_trans[temp_trans['days'] < days]

  g['churned_users']= g.apply(func, axis=1)
  g.head(10)
  
  for x in range(0,len(g)):
    if(g['churned_users'][x] == 1):
      c[q] += 1
  #print(a)
  days = days - 90
  print(temp.shape)

425
(1916387, 13)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

(36117, 13)
(178555, 13)
(479161, 13)
(947120, 13)
(1578776, 13)
(1916387, 13)


In [783]:
#Total number of NEW customers per quarter year
u = temp_users

u.created_date=pd.to_datetime(u.created_date)
u['days'] = (np.datetime64('2019-06-01') - u['created_date']).dt.days

days = np.max(u['days']) 
t = [0,0,0,0,0,0]
for x in range(0,len(u)):
    t[u['days'][x]//90] += 1

print(t) #t[0] is the most recent, t[5] is the oldest

[0, 1810, 4449, 3862, 3367, 2056]


In [784]:
counter=0
for i in range (0,len(t)):
  counter += t[i]
counter 

15544

In [785]:
t_sum= [0,0,0,0,0,0]
for i in range (1,len(t)):
  t_sum[i] = t[i] + t_sum[i-1]
t_sum #t_sum[0] is the most recent, t_sum[5] is the oldest

[0, 1810, 6259, 10121, 13488, 15544]

In [820]:
#Churn rate per quarter year
churn_rate= [0,0,0,0,0,0]
for i in range (1,len(churn_rate)): # gia to 1o 3mhno den mporoyme na exoume provlepsi
  churn_rate[i] = format( (100 * c[i]) / (t_sum[i]) , '.10f') #aplh methodos twn triwn 

print(churn_rate)

[0, '33.2044198895', '13.7402140917', '13.2793202253', '14.7019572954', '19.1585177560']


In [0]:
#t_rev= [0,0,0,0,0,0]
#for i in range (0,len(t)):
#  t_rev[i] = t[len(t)-1-i]

In [0]:
#t_rev = Total number of new customers per quarter year
#t_rev

In [788]:
#c = churn users per quarter year
c 

[478, 601, 860, 1344, 1983, 2978]

In [789]:
#a[0] = FAILED notifications per quarter year
a[0]

[2061, 12865, 3658, 3509, 3829, 89]

In [790]:
#a[1] = SENT notifications per quarter year
a[1]

[6747, 36130, 18498, 5937, 2868, 1513]

In [791]:
#Total number of customers per quarter year
t_sum

[0, 1810, 6259, 10121, 13488, 15544]

In [822]:
#Churn rate per quarter year
print(churn_rate)

[0, '33.2044198895', '13.7402140917', '13.2793202253', '14.7019572954', '19.1585177560']
